In [1]:
import numpy as np
import webbrowser
from aocd.models import Puzzle
import re
from dataclasses import dataclass
import json
from collections import defaultdict
from typing import Optional
%matplotlib inline
puzzle = Puzzle(year=2021, day=18)

In [2]:
webbrowser.open(puzzle.url);

## Part 1

In [2]:
input_data = """
[1,2]
[[1,2],3]
[9,[8,7]]
[[1,9],[8,5]]
[[[[1,2],[3,4]],[[5,6],[7,8]]],9]
[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]
[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]
""".strip()

In [7]:
@dataclass
class Node:
    left: "Node"
    right: "Node"
    parent: Optional["Node"] = None

@dataclass
class Val:
    val: int
    parent: Optional[Node] = None
    left_ngbr: Optional["Val"] = None
    right_ngbr: Optional["Val"] = None
    
    def __str__(self):
        return str(self.val)
    
    def __repr__(self):
        return str(self)

In [8]:
def tree_from_input(line):
    data = json.loads(line)
    return _generate_tree(data)
    
def _generate_tree(data):
    if isinstance(data, int):
        return Val(data)
    else:
        assert len(data) == 2
        return Node(_generate_tree(data[0]), _generate_tree(data[1]))

In [9]:
def traverse_tree(tree, depth: int = 0):
    yield tree, depth
    if isinstance(tree, Node):
        yield from traverse_tree(tree.left, depth + 1)
        yield from traverse_tree(tree.right, depth + 1)

In [10]:
def print_tree(tree):
    out = ""
    if isinstance(tree, Node):
        out += "[" + print_tree(tree.left) + "," + print_tree(tree.right) + "]"
    else:
        out += str(tree.val)
    return out

In [17]:
def _assign_neighbours(tree: Node):
    prev = None
    for item, _ in traverse_tree(tree):
        if isinstance(item, Val):
            item.left_ngbr = prev
            if prev is not None:
                prev.right_ngbr = item
            prev = item


def _assign_parents(tree, parent = None):
    tree.parent = parent
    if isinstance(tree, Node):
        _assign_parents(tree.left, tree)
        _assign_parents(tree.right, tree)

def assign(tree):
    _assign_neighbours(tree)
    _assign_parents(tree)

In [18]:
def explode_action(tree):
    for node, depth in traverse_tree(tree):
        if isinstance(node, Val) and depth >= 5:
            pair = node.parent
            left, right = pair.left, pair.right
            # we are only interested in pairs where both the left and right are already values and no further nodes
            if not isinstance(left, Val) or not isinstance(right, Val):
                continue
            if left.left_ngbr is not None:
                left.left_ngbr.val += left.val
            if right.right_ngbr is not None:
                right.right_ngbr.val += right.val
            
            if pair.parent.left == pair:
                pair.parent.left = Val(0)
            else:
                pair.parent.right = Val(0)
            
            return True  # exploded something
    
    return False  # nothing exploded

### Some tests for explode

In [19]:
explode_tests = {
    "[[[[[9,8],1],2],3],4]": "[[[[0,9],2],3],4]",
    "[7,[6,[5,[4,[3,2]]]]]": "[7,[6,[5,[7,0]]]]",
    "[[6,[5,[4,[3,2]]]],1]": "[[6,[5,[7,0]]],3]",
    "[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]": "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]",
    "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]": "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"
}

for test_in, test_out in explode_tests.items():
    tree = tree_from_input(test_in)
    assign(tree)
    assert explode_action(tree)
    assert print_tree(tree) == test_out

In [24]:
def split_action(tree):
    for node, _ in traverse_tree(tree):
        if isinstance(node, Val) and node.val >= 10:
            new_node = Node(Val(node.val // 2), Val((node.val + 1) // 2))
            if node.parent.left == node:
                node.parent.left = new_node
            else:
                node.parent.right = new_node
            return True  # splitted something
    return False  # nothing splitted

In [28]:
def add(trees):
    result = trees[0]
    for tree in trees[1:]:
        result = add2(result, tree)
    return result

def add2(tree1, tree2):
    tree = Node(tree1, tree2)
    tree = reduce(tree)
    return tree

def reduce(tree):
    while True:
        assign(tree)
        if explode_action(tree):  # something exploded, check if we can explode again
            continue
        elif split_action(tree):  # nothing exploded, check if we can explode / split again
            continue
        else:  # finished
            return tree

In [29]:
add_tests = {
    """
[[[[4,3],4],4],[7,[[8,4],9]]]
[1,1]
    """.strip(): "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]",
    """
[1,1]
[2,2]
[3,3]
[4,4]
    """.strip(): "[[[[1,1],[2,2]],[3,3]],[4,4]]",
    """
[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
    """.strip(): "[[[[3,0],[5,3]],[4,4]],[5,5]]",
    """
[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]
    """.strip(): "[[[[5,0],[7,4]],[5,5]],[6,6]]",
"""
[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]
""".strip(): "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"
}

In [31]:
for test_in, test_out in add_tests.items():
    test_lines = test_in.split("\n")
    test_trees = [tree_from_input(l) for l in test_lines]
    test_result = add(test_trees)
    assert print_tree(test_result) == test_out

In [33]:
def magnitude(tree):
    if isinstance(tree, Node):
        return 3*magnitude(tree.left) + 2*magnitude(tree.right)
    return tree.val

In [37]:
test_magnitude = {
    "[[1,2],[[3,4],5]]": 143,
    "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]": 3488
}

In [38]:
for test_in, test_out in test_magnitude.items():
    assert magnitude(tree_from_input(test_in)) == test_out

### Now for the actual input

In [47]:
input_data = puzzle.input_data
lines = input_data.split("\n")
trees = [tree_from_input(l) for l in lines]

In [48]:
result_tree = add(trees)
result = magnitude(result_tree)
result

3675

In [49]:
puzzle.answer_a = result

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


## Part 2

In [65]:
from itertools import combinations
from tqdm.auto import tqdm

In [62]:
pairs = list(combinations(lines, 2))
len(pairs)  # check how many combinations we need to check

4950

In [71]:
largest_mag = -np.inf

def calc_result_magnitude(a, b):
    tree = add([tree_from_input(a), tree_from_input(b)])
    return magnitude(tree)

for a, b in tqdm(pairs):
    m = calc_result_magnitude(a, b)
    if m > largest_mag:
        largest_mag = m
    
    m = calc_result_magnitude(b, a)
    if m > largest_mag:
        largest_mag = m

  0%|          | 0/4950 [00:00<?, ?it/s]

In [72]:
result = largest_mag
result

4650

In [73]:
puzzle.answer_b = result

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 18! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
